In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw, drop_invalid=True)
dev_pack_processed = preprocessor.transform(dev_pack_raw, drop_invalid=False)
test_pack_processed = preprocessor.transform(test_pack_raw, drop_invalid=False)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3723.69it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2360.95it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1800464.76it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3658.44it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2354.42it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 33863.15it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 4006.84it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7ff3dc3c3be0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7ff3654f4ef0>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'input_shapes': [(10,), (100,)]}

In [7]:
model = mz.models.DRMMTKS()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['mask_value'] = -1
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['optimizer'] = 'adadelta'

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.drmmtks.DRMMTKS'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adadelta
with_embedding                True
embedding_input_dim           16675
embedding_output_dim          300
embedding_trainable           True
with_multi_layer_perceptron   True
mlp_num_units                 5
mlp_num_layers                1
mlp_num_fan_out               1
mlp_activation_func           relu
mask_value                    -1
top_k                         20


In [8]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 10, 100)      0           embedding[0][0]                  
          

In [9]:
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [10]:
model.load_embedding_matrix(embedding_matrix)

In [11]:
test_x, test_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_x))

In [12]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [13]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=4, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 11s 107ms/step - loss: 0.9117
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5719898092987374 - normalized_discounted_cumulative_gain@5(0.0): 0.6239251359887107 - mean_average_precision(0.0): 0.5905114616764597
Epoch 2/30
102/102 [==============================] - 14s 142ms/step - loss: 0.5512
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.61774368985056 - normalized_discounted_cumulative_gain@5(0.0): 0.6774292390410019 - mean_average_precision(0.0): 0.6311750071640536
Epoch 3/30
102/102 [==============================] - 14s 139ms/step - loss: 0.3861
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.647813998446898 - normalized_discounted_cumulative_gain@5(0.0): 0.6985697843928311 - mean_average_precision(0.0): 0.6491233856760419
Epoch 4/30
102/102 [==============================] - 14s 136ms/step - loss: 0.2907
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6495566940134582 - normal

Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6031136465042117 - normalized_discounted_cumulative_gain@5(0.0): 0.6520833457413675 - mean_average_precision(0.0): 0.6070763488486232


In [14]:
append_params_to_readme(model)